# Hypothesis Testing on INDIAN KIDS SCREEN TIME
**GUNGUN JAIN T085**

In this notebook we will reuse the ideas from the original **Hypothesis_Testing.ipynb**,  
but instead of dummy data we will apply all tests on a real dataset:  
**`INDIAN KIDS SCREEN TIME.csv`** (all lines from the show).

We will perform and interpret:

1. One-sample t-test  
2. Two-sample independent t-test  
3. Paired t-test  
4. Chi-square test of independence  
5. One-way ANOVA (F-test)  
6. Two-way ANOVA  

All examples will be based on meaningful questions about the dialogue data.


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols

sns.set_theme()

# Load dataset
df = pd.read_csv("/content/Indian_Kids_Screen_Time.csv")
print("Shape:", df.shape)

# Show sample rows (same as iloc[8:20])
df.iloc[8:20]


Shape: (9712, 8)


,Age,Gender,Avg_Daily_Screen_Time_hr,Primary_Device,Exceeded_Recommended_Limit,Educational_to_Recreational_Ratio,Health_Impacts,Urban_or_Rural
8,14,Male,4.61,Laptop,True,0.36,"Poor Sleep, Anxiety",Rural
9,18,Male,3.24,Tablet,True,0.48,"Poor Sleep, Obesity Risk",Urban
10,18,Male,3.53,Tablet,True,0.44,Poor Sleep,Rural
11,15,Female,4.94,TV,True,0.36,Poor Sleep,Rural
12,12,Male,4.59,Smartphone,True,0.43,Poor Sleep,Urban
13,11,Male,6.08,TV,True,0.48,Eye Strain,Rural
14,15,Male,6.15,Smartphone,True,0.40,Poor Sleep,Urban
15,15,Female,3.69,Smartphone,True,0.34,Poor Sleep,Urban
16,10,Female,7.10,Smartphone,True,0.43,"Poor Sleep, Eye Strain",Rural
17,13,Female,6.98,Smartphone,True,0.48,Obesity Risk,Urban


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9712 entries, 0 to 9711
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                9712 non-null   int64  
 1   Gender                             9712 non-null   object 
 2   Avg_Daily_Screen_Time_hr           9712 non-null   float64
 3   Primary_Device                     9712 non-null   object 
 4   Exceeded_Recommended_Limit         9712 non-null   bool   
 5   Educational_to_Recreational_Ratio  9712 non-null   float64
 6   Health_Impacts                     6494 non-null   object 
 7   Urban_or_Rural                     9712 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 540.7+ KB


In [ ]:
# Keep only rows where screen time value exists
df['Avg_Daily_Screen_Time_hr'] = df['Avg_Daily_Screen_Time_hr'].astype(str)

# Some rows have 'nan' as string because of astype(str); treat those as missing
df['screen_time_clean'] = df['Avg_Daily_Screen_Time_hr'].replace('nan', np.nan)

# Convert screen time into a numeric value (hours per day)
df['screen_time_numeric'] = pd.to_numeric(df['screen_time_clean'], errors='coerce')

# Indicator: does this row contain valid screen time?
df['has_screen_time'] = ~df['screen_time_clean'].isna()

# Age group factor for ANOVA: younger vs older kids
df['age_group'] = np.where(df['Age'] <= df['Age'].median(), 'younger', 'older')

df[['Age','Gender','screen_time_clean','screen_time_numeric','has_screen_time','age_group']].head()


,Age,Gender,screen_time_clean,screen_time_numeric,has_screen_time,age_group
0,14,Male,3.99,3.99,True,older
1,11,Female,4.61,4.61,True,younger
2,18,Female,3.73,3.73,True,older
3,15,Female,1.21,1.21,True,older
4,12,Female,5.89,5.89,True,younger


1. One-Sample t-test

Goal: To test whether the average daily screen time of children is equal to a chosen reference value.

Example hypothesis:

Null hypothesis (H₀):
The mean daily screen time of children is 5 hours.

Alternative hypothesis (H₁):
The mean daily screen time of children is different from 5 hours.

In [ ]:
# Drop missing screen time values
lengths = df['screen_time_numeric'].dropna()

print("Number of valid screen time entries:", len(lengths))
print("Sample mean daily screen time (hours):", lengths.mean())

# Perform one-sample t-test against population mean = 5 hours
t_stat, p_value = stats.ttest_1samp(lengths, popmean=5.0)

print("\nOne-sample t-test for mean screen time = 5 hours")
print("t-statistic:", t_stat)
print("p-value:", p_value)


Number of valid screen time entries: 9712
Sample mean daily screen time (hours): 4.3528366968698515

One-sample t-test for mean screen time = 5 hours
t-statistic: -37.11814815010821
p-value: 3.910422182691961e-282


Interpretation:

If the p-value is less than 0.05, we reject H₀ and conclude that
the average daily screen time of children is significantly different from 5 hours.

If the p-value is greater than or equal to 0.05, we do not have enough evidence to say that
the true mean daily screen time differs from 5 hours.

This test uses the actual screen time values from the dataset,
so the result reflects how many hours children typically spend on screens each day.

2. Two-Sample Independent t-test

Now we compare daily screen time between Urban and Rural children.

Example hypothesis:

H₀ (Null hypothesis):
The mean daily screen time of Urban and Rural children is the same.

H₁ (Alternative hypothesis):
The mean daily screen time of Urban and Rural children is different.

In [ ]:
# Filter for Urban and Rural only
urban = df[(df['Urban_or_Rural'] == "Urban") & df['screen_time_numeric'].notna()]['screen_time_numeric']
rural = df[(df['Urban_or_Rural'] == "Rural") & df['screen_time_numeric'].notna()]['screen_time_numeric']

print("Urban kids: count:", len(urban), "mean screen time:", urban.mean())
print("Rural kids: count:", len(rural), "mean screen time:", rural.mean())

# Two-sample t-test with unequal variances (Welch's t-test)
t_stat, p_value = stats.ttest_ind(urban, rural, equal_var=False)

print("\nTwo-sample t-test: Urban vs Rural daily screen time")
print("t-statistic:", t_stat)
print("p-value:", p_value)


Urban kids: count: 6851 mean screen time: 4.344123485622537
Rural kids: count: 2861 mean screen time: 4.373701502970989

Two-sample t-test: Urban vs Rural daily screen time
t-statistic: -0.76837668436743
p-value: 0.4422977761711223


Interpretation

A small p-value (< 0.05) suggests that Urban and Rural children have significantly different average daily screen time.

A large p-value (≥ 0.05) suggests that any observed difference in mean screen time could simply be due to random variation.

This tells us whether children’s screen-time habits differ based on whether they live in urban or rural areas.

3. Paired t-test

A paired t-test compares two related measurements for the same child.

We construct a meaningful pairing as follows:

For each child, compute:

Educational screen time per day

Recreational screen time per day

These two measurements are naturally linked because they belong to the same child and sum up to the total daily screen time.

We test:

H₀:
There is no difference in mean educational and recreational daily screen time.

H₁:
There is a difference in how much time children spend on educational vs recreational usage.

This test helps us understand whether kids tend to spend more time learning or more time on entertainment when using screens.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Load dataset
df = pd.read_csv("Indian_Kids_Screen_Time.csv")

# Restrict to rows with non-missing screen time
s1_df = df[df['Avg_Daily_Screen_Time_hr'].notna()].copy()

group_first = []
group_second = []

# Group by Primary_Device (similar to episodes)
for grp, grp_df in s1_df.groupby('Primary_Device'):
    grp_df = grp_df.sort_values('Avg_Daily_Screen_Time_hr')
    n = len(grp_df)

    if n < 4:
        continue

    mid = n // 2
    first_half = grp_df['Avg_Daily_Screen_Time_hr'].iloc[:mid].mean()
    second_half = grp_df['Avg_Daily_Screen_Time_hr'].iloc[mid:].mean()

    group_first.append(first_half)
    group_second.append(second_half)

group_first = np.array(group_first)
group_second = np.array(group_second)

print("Number of device-groups used:", len(group_first))
print("Mean screen time (first half):", group_first.mean())
print("Mean screen time (second half):", group_second.mean())

# Paired t-test
t_stat, p_value = stats.ttest_rel(group_first, group_second)

print("\nPaired t-test: first half vs second half within each device group")
print("t-statistic:", t_stat)
print("p-value:", p_value)


Number of device-groups used: 4
Mean screen time (first half): 3.0383237020671077
Mean screen time (second half): 5.641932393401432

Paired t-test: first half vs second half within each device group
t-statistic: -9.801164996487689
p-value: 0.002257340796415978


Interpretation

A low p-value (< 0.05) suggests that the average educational screen time is significantly different from the average recreational screen time for the same children.

A high p-value (≥ 0.05) indicates that we do not have strong evidence of a difference between the two types of screen usage.

This helps us understand whether children tend to spend more time on learning activities or on entertainment when using screens.

4. Chi-square Test of Independence (Modified for Your Dataset)

The Chi-square test checks whether two categorical variables are independent.

In this dataset, we test whether exceeding recommended daily screen time
depends on whether a child lives in an urban or rural area.

Variables:

Variable 1: Urban_or_Rural
(Urban vs Rural children)

Variable 2: Exceeded_Recommended_Limit
(True = child exceeds recommended screen time, False = does not)

Hypotheses:

H₀:
Urban_or_Rural and Exceeded_Recommended_Limit are independent
(place of living does not affect whether a child exceeds screen-time limits).

H₁:
Urban_or_Rural and Exceeded_Recommended_Limit are associated
(urban or rural setting does influence whether a child exceeds screen-time limits).

In [ ]:
from scipy.stats import chi2_contingency

# Build contingency table
contingency = pd.crosstab(df['Primary_Device'], df['Exceeded_Recommended_Limit'])
print("Contingency table:", contingency)

chi2_stat, p_value, dof, expected = chi2_contingency(contingency)

print("\nChi-square test of independence: Primary_Device vs Exceeded_Recommended_Limit")
print("Chi-square statistic:", chi2_stat)
print("Degrees of freedom:", dof)
print("p-value:", p_value)


Contingency table: Exceeded_Recommended_Limit  False  True 
Primary_Device                          
Laptop                        158   1275
Smartphone                    619   3949
TV                            402   2085
Tablet                        232    992

Chi-square test of independence: Primary_Device vs Exceeded_Recommended_Limit
Chi-square statistic: 42.33917841275176
Degrees of freedom: 3
p-value: 3.3993201695284536e-09


Interpretation

If the p-value is small (< 0.05), we reject independence and conclude that
Urban vs Rural children differ in whether they exceed recommended screen-time limits.

If the p-value is large (≥ 0.05), then
Urban and Rural kids have similar proportions of children exceeding the recommended screen-time limit.

This helps us understand whether a child’s living environment (urban or rural)
is associated with their likelihood of exceeding healthy screen-time guidelines.

5. One-way ANOVA (F-test)

ANOVA compares the means of more than two groups.
Here we test whether the mean daily screen time is the same across all device types children use.

Groups:

Mobile, Tablet, Laptop, TV, etc.

Response variable:

screen_time_numeric (daily screen time in hours)

Hypotheses:

H₀:
All device groups have the same mean daily screen time.

H₁:
At least one device group has a different mean daily screen time.

In [ ]:
from scipy import stats

# Collect screen time values for each device type
device_groups = []
labels = []

for d, g in df[df['Avg_Daily_Screen_Time_hr'].notna()].groupby('Primary_Device'):
    device_groups.append(g['Avg_Daily_Screen_Time_hr'].values)
    labels.append(f"Device: {d}")

print("Device groups included:", labels)

F_stat, p_value = stats.f_oneway(*device_groups)

print("\nOne-way ANOVA: Screen time across device types")
print("F-statistic:", F_stat)
print("p-value:", p_value)


Device groups included: ['Device: Laptop', 'Device: Smartphone', 'Device: TV', 'Device: Tablet']

One-way ANOVA: Screen time across device types
F-statistic: 5.919665342027592
p-value: 0.0004967622706483381


Interpretation

A small p-value (< 0.05) suggests that at least one device type has a
significantly different average daily screen time compared to the others.

A large p-value (≥ 0.05) indicates that the data does not provide strong evidence
that children using different devices have different average screen-time levels.

ANOVA is a generalisation of the two-sample t-test for comparing more than two groups.

6. Two-way ANOVA
Two-way ANOVA allows us to examine the effects of two categorical factors on a continuous dependent variable, along with their interaction.

For this dataset:

Factor A: Age_Group
(e.g., 3–5 yrs, 6–8 yrs, 9–12 yrs)

Factor B: Device_Type
(e.g., Mobile, Tablet, TV, Laptop)

Response Variable: Screen_Time
(daily screen time in minutes/hours)

Research Questions

Does Age Group affect average screen time?

Does Device Type affect screen time?

Is there an interaction between Age Group and Device Type?
→ i.e., does the effect of device type vary across age groups?

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Create a clean subset with non-missing screen time
anova_df = df[['Avg_Daily_Screen_Time_hr', 'Age', 'Primary_Device']].dropna().copy()

# Convert factors to categorical
anova_df['Age'] = anova_df['Age'].astype('category')
anova_df['Primary_Device'] = anova_df['Primary_Device'].astype('category')

# Fit two-way ANOVA model with interaction
model = ols('Avg_Daily_Screen_Time_hr ~ C(Age) * C(Primary_Device)', data=anova_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

anova_table


/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '


,sum_sq,df,F,PR(>F)
C(Age),3178.214778,10.0,110.104718,9.709600e-177
C(Primary_Device),4.146202,3.0,0.478797,6.970343e-01
C(Age):C(Primary_Device),81.696855,30.0,0.943424,5.544226e-01
Residual,27915.711269,9671.0,NaN,NaN


Interpretation:

From the ANOVA table:

The row for C(Age_Group) tests whether mean screen time differs across age groups.

The row for C(Device_Type) tests whether screen time differs across different device types.

The row for C(Age_Group):C(Device_Type) tests whether the effect of device type
depends on the age group (interaction).

For each row, a small p-value indicates a statistically significant effect.

Summary

In this notebook we:

Used real screen-time data from Indian kids instead of artificial numbers.

Applied classic hypothesis tests (t-tests, Chi-square, ANOVA) to answer
questions about how screen time varies across age groups and device types.

Demonstrated both numerical results (test statistics and p-values) and
clear verbal interpretations for each test.

You can now modify the hypotheses (for example, compare specific age groups or device types)
and re-run the same tests to explore other questions about children’s screen-time behavior.